<a href="https://colab.research.google.com/github/adapter001/Pine/blob/main/Supertrend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Supertrend script is a versatile tool for detecting and visualizing trends, generating actionable buy/sell signals, and supporting alert-driven trading strategies. It’s particularly effective for trending markets but requires careful parameter tuning and additional tools for optimal performance.

In [ ]:
//@version=5
indicator('Supertrend', overlay=true, format=format.price, precision=2)

// Inputs
Periods = input.int(title='ATR Period', defval=10)
src = input.source(hl2, title='Source')  // Input source
Multiplier = input.float(title='ATR Multiplier', step=0.1, defval=3.0)
changeATR = input.bool(title='Change ATR Calculation Method?', defval=true)
showsignals = input.bool(title='Show Buy/Sell Signals?', defval=true)
highlighting = input.bool(title='Highlighter On/Off?', defval=true)

// ATR Calculation
atr = changeATR ? ta.atr(Periods) : ta.sma(ta.tr, Periods)

// Supertrend Calculation
up = src - Multiplier * atr
up1 = nz(up[1], up)
up := close[1] > up1 ? math.max(up, up1) : up

dn = src + Multiplier * atr
dn1 = nz(dn[1], dn)
dn := close[1] < dn1 ? math.min(dn, dn1) : dn

trend = 1
trend := nz(trend[1], trend)
trend := trend == -1 and close > dn1 ? 1 : trend == 1 and close < up1 ? -1 : trend

// Plotting Supertrend Lines
upPlot = plot(trend == 1 ? up : na, title='Up Trend', style=plot.style_line, linewidth=2, color=color.new(color.green, 0))
dnPlot = plot(trend == -1 ? dn : na, title='Down Trend', style=plot.style_line, linewidth=2, color=color.new(color.red, 0))

// Buy and Sell Signals
buySignal = trend == 1 and trend[1] == -1
sellSignal = trend == -1 and trend[1] == 1

plotshape(buySignal and showsignals ? up : na, title='Buy', location=location.absolute, style=shape.labelup, text='Buy', size=size.tiny, color=color.new(color.green, 0), textcolor=color.new(color.white, 0))
plotshape(sellSignal and showsignals ? dn : na, title='Sell', location=location.absolute, style=shape.labeldown, text='Sell', size=size.tiny, color=color.new(color.red, 0), textcolor=color.new(color.white, 0))

// Highlighting the Trend
mPlot = plot(ohlc4, title='Price Midpoint', style=plot.style_circles, linewidth=1)
longFillColor = highlighting and trend == 1 ? color.rgb(0, 255, 0, 90) : na
shortFillColor = highlighting and trend == -1 ? color.rgb(255, 0, 0, 90) : na

fill(mPlot, upPlot, title='UpTrend Highlighter', color=longFillColor)
fill(mPlot, dnPlot, title='DownTrend Highlighter', color=shortFillColor)

// Alert Conditions
alertcondition(buySignal, title='SuperTrend Buy', message='SuperTrend Buy!')
alertcondition(sellSignal, title='SuperTrend Sell', message='SuperTrend Sell!')
changeCond = trend != trend[1]
alertcondition(changeCond, title='SuperTrend Direction Change', message='SuperTrend has changed direction!')
